In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from collections import Counter
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/levbarbash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1. Load and merge the datasets keeping all information available for the dates in
which there is a measurement in “fx.csv”

In [2]:
speeches = pd.read_csv("speeches.csv", sep='|')
speeches = speeches[speeches['date'] >= '1999-01-04']
fx = pd.read_csv("fx.csv", sep=',')
fx = fx[['TIME_PERIOD', 'OBS_VALUE']]
merged = speeches.merge(fx, how='left', left_on='date', right_on='TIME_PERIOD').drop(columns=['TIME_PERIOD'])

3. Handle missing observations for the exchange rate, if any. This should be done
replacing any missing exchange rate with the latest information available.
Whenever this cannot be done, the relevant entry should be removed entirely
from the dataset

In [3]:
merged['was_null'] = 0
merged.loc[merged['OBS_VALUE'].isnull(), 'was_null'] = 1
old_dates = merged.loc[merged['was_null'] == 1, 'date']
while merged['OBS_VALUE'].isnull().any():
    merged.loc[merged['OBS_VALUE'].isnull(), 'date'] = (pd.to_datetime(merged[merged['OBS_VALUE'].isnull()]['date']) 
                                                        - timedelta(days=1)).astype(str)
    merged = merged.drop(columns=['OBS_VALUE'])
    merged = merged.merge(fx, how='left', left_on='date', right_on='TIME_PERIOD').drop(columns=['TIME_PERIOD'])
merged.loc[merged['was_null'] == 1, 'date'] = old_dates

4. Calculate the exchange rate return. Extend the original dataset with the
following variables: “good_news” (equal to 1 when the exchange rate return is
larger than 0.5 percent, 0 otherwise) and “bad_news” (equal to 1 when the
exchange rate return is lower than -0.5 percent, 0 otherwise)

In [4]:
prev_val = merged['OBS_VALUE'][1:].reset_index(drop=True)
merged['prev_OBS_VALUE'] = prev_val
merged.loc[merged['prev_OBS_VALUE'].isnull(), 'prev_OBS_VALUE'] = fx[fx['TIME_PERIOD'] == '1999-01-13']['OBS_VALUE'].reset_index(drop=True)[0]
merged['return'] = (merged['OBS_VALUE'] - merged['prev_OBS_VALUE']) / (merged['prev_OBS_VALUE'] / 100)
merged['good_news'] = 0
merged['bad_news'] = 0
merged.loc[merged['return'] >= 0.5, 'good_news'] = 1
merged.loc[merged['return'] <= -0.5, 'bad_news'] = 1

5. Remove the entries for which contents column has NA values. Generate and
store in csv the following tables:

a. “good_indicators” – with the 20 most common words (excluding articles,
prepositions and similar connectors) associated with entries wherein
“good_news” is equal to 1;

In [5]:
from collections import Counter
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')
merged = merged.dropna(subset=['contents'])
merged['contents'] = merged['contents'].str.lower()
good_inds = Counter(" ".join(merged[merged['good_news'] == 1]["contents"]).split()).most_common()
good_inds_df = pd.DataFrame(good_inds, columns=['word', 'count'])
stop_words = pd.Series(list(set(stopwords.words('english'))))
stop_words = pd.concat([stop_words, pd.Series(['de', '–', 'la', 'also', 'der', 'may'])], ignore_index=True)
stop_words_df = pd.DataFrame(stop_words, columns=['word'])
stop_words_df['is_stop_word'] = 1
merged_good_inds = good_inds_df.merge(stop_words_df, how='left', left_on='word', right_on='word')
merged_good_inds[merged_good_inds['is_stop_word'] != 1].drop(columns=['is_stop_word']).head(20)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/levbarbash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,word,count
14,euro,8185
15,financial,7659
20,monetary,6641
21,policy,6067
32,area,4317
33,economic,4267
34,central,4261
39,european,3701
40,market,3665
41,would,3336


b. “bad_indicators” – with the 20 most common words (excluding articles,
prepositions and similar connectors) associated with entries wherein
“bad_news” is equal to 1

In [6]:
bad_inds = Counter(" ".join(merged[merged['bad_news'] == 1]["contents"]).split()).most_common()
bad_inds_df = pd.DataFrame(bad_inds, columns=['word', 'count'])
merged_bad_inds = bad_inds_df.merge(stop_words_df, how='left', left_on='word', right_on='word')
merged_bad_inds[merged_bad_inds['is_stop_word'] != 1].drop(columns=['is_stop_word']).head(20)

,word,count
14,euro,8733
18,financial,7842
20,monetary,7140
23,policy,6158
31,area,4969
32,economic,4795
34,central,4389
36,european,4198
39,market,3890
40,would,3809
